# 07 Assistants API | 01 Assistant Run Thread

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_DemoEnvironment/01_Environment.ipynb))

## Step 1: Create Assistants Client

The AssistantsClient from the NuGet package Azure.AI.OpenAI.Assistants serves as an interface for developers to interact with OpenAI's Assistants API, facilitating the integration of AI-powered conversational agents into applications. Through this client, developers can send queries to AI assistants, receive responses, and manage conversation sessions, enabling seamless deployment of virtual assistants for various tasks such as customer support, information retrieval, and task automation.

The AssistantsClient from Azure.AI.OpenAI.Assistants is a .NET client library that acts as the centralized point for all .NET functionality that want to interact with a deployed Azure OpenAI Large Language Model using the Assistants API. It provides all methods to access the OpenAI Assistants REST API for various tasks such as Assistant creation, Thread creation, Run creation etc.. 

The OpenAIClient can connect to LLMs supporting the Assistants API. 

In [ ]:
#r "nuget: Azure.AI.OpenAI.Assistants, 1.0.0-beta.3"
#r "nuget: DotNetEnv, 2.5.0"

using Azure;
using Azure.AI.OpenAI.Assistants;
using DotNetEnv;

//configuration file is created during environment creation
//if you skipped the deployment just remove the code and provide values from your deployment
static string _configurationFile = @"../01_DemoEnvironment/conf/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("SKIT_AOAI1106_APIKEY") ?? "SKIT_AOAI1106_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("SKIT_AOAI1106_ENDPOINT") ?? "SKIT_AOAI1106_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("SKIT_AOAI1106_DEPLOYMENTNAME") ?? "SKIT_AOAI1106_DEPLOYMENTNAME not found";

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);
AssistantsClient assistantsClient = new AssistantsClient(new Uri(oAiEndpoint), azureKeyCredential);

Console.WriteLine($"AssistantClient created...");


Expected output:

```
Installed Packages
- Azure.AI.OpenAI.Assistants, 1.0.0-beta.3
- DotNetEnv, 2.5.0

AssistantClient created...
```

## Step 2: Create Assistant

The following cell, demonstrate the creation of a basic Assistant. To create the Assistant the following information is provided: 

- Assistant name: A user friendly name for the to be created Assistant
- Assistant instructions: The system message or system prompt which defines the Assistant.
- Assistant description: A user friendly description of the to be created Assistant

The `CreateAssistantAsync()` function call returns a unique id of the created Assistant which is used for all further interaction with the Assistant.

In [ ]:
string assistantName = "SimpleAssistant";
string assistantInstruction = @"
    You help people find information and answer questions.
";
string assistantDescription = "Assistant to help answer questions based on LLM training data";

AssistantCreationOptions assistantCreationOptions; 
assistantCreationOptions = new AssistantCreationOptions(chatCompletionDeploymentName){
    Name = assistantName,
    Description = assistantDescription,
    Instructions = assistantInstruction,
}; 

Response<Assistant> responseAssistant = await assistantsClient.CreateAssistantAsync(assistantCreationOptions);

Console.WriteLine($"Assistant created: {responseAssistant.Value.Id}");


Expected outcome:

```
Assistant created: asst_xxxxxxxxxxxxxx
```

## Step 3: Create Thread

A Thread is used as container for user messages as well as LLM creates assistant messages. It stores the whole chat history within the server side LLM. The function call `CreateThreadAsync()` returns a unique Thread id which is used for all further interactions with the Thread.

In [ ]:
Response<AssistantThread> responseThread = await assistantsClient.CreateThreadAsync();

Console.WriteLine($"Thread created: {responseThread.Value.Id}");

Expected outcome:

```
Thread created: thread_xxxxxxxxxxxxxxxxxxxx
```

## Step 4: Add Message to Thread

As mentioned in step 3 LLM chat history is stored within a Thread. This means to provide a "question" to the LLM it needs to be stored within the created Thread. This is done calling the function `CreateMessageAsync()` where the user message, the Thread id as well as `MessageRole.User`is provided.

In [ ]:
string messageContent = "What are the 3 biggest cloud provider based on regional presence?";
        
Response<ThreadMessage> responseThreadMessage = await assistantsClient.CreateMessageAsync(
    responseThread.Value.Id, 
    MessageRole.User, 
    messageContent
);

Console.WriteLine($"Message created: {responseThreadMessage.Value.Id}");

Expected outcome:

```
Message created: msg_xxxxxxxxxxxxxxxxxx
```

## Step 5: Create Run

After an Assistant and a Thread with user messages has been created both abstractions can be brought together in a so called RunThread to perform with the help of an LLM instance the completion. Which can be somehow described as "answering the user question". The Azure OpenAI instance to be used has been provided while creating the `AssistantsClient` object while the specific LLM deployment has been provided while creating the Assistants object. 

The function call `CreateRunAsync()` can be used to create the asynchronous Run. It provides a unique Run id which can be used to retrieve the state of the asynchronous execution. 

In [ ]:
CreateRunOptions createRunOptions = new CreateRunOptions(responseAssistant.Value.Id); 

Response<ThreadRun> responseThreadRun = await assistantsClient.CreateRunAsync(
    responseThread.Value.Id, 
    createRunOptions
); 

Console.WriteLine($"Run created: {responseThreadRun.Value.Id}");

Expected outcome:

```
Run created: run_xxxxxxxxxxxxxxxxxxxxxxxx
```

## Step 6: Check Status & Retrieve Message

The sample cell waits for 5 seconds to allow some time for completion. It uses: 

- `GetRunAsync()`: to retrieve the current state of the Run
- `GetMessagesAsync()`: to retrieve **all** messages from the Thread

and shows the latest message from Thread which is the LLM completion


In [ ]:
//Give some time to finalize Run
System.Threading.Thread.Sleep(TimeSpan.FromSeconds(5));

//Get Run status
responseThreadRun = await assistantsClient.GetRunAsync(
    responseThread.Value.Id,
    responseThreadRun.Value.Id
); 

Console.WriteLine($"Run status: {responseThreadRun.Value.Status}");

//Wait for Run completion
while(responseThreadRun.Value.Status != RunStatus.Completed) {
    Console.WriteLine($"Waiting for run completion...");
    System.Threading.Thread.Sleep(TimeSpan.FromSeconds(5));
    responseThreadRun = await assistantsClient.GetRunAsync(
        responseThread.Value.Id,
        responseThreadRun.Value.Id
    ); 
}

//Get Run results
Response<PageableList<ThreadMessage>> responseThreadMessages = await assistantsClient.GetMessagesAsync(
    responseThread.Value.Id
);

//Get last assistnat created message
ThreadMessage threadMessage = responseThreadMessages.Value.First<ThreadMessage>();

Console.WriteLine($"Message created by/at: {threadMessage.Role} - {threadMessage.CreatedAt}");
foreach(MessageContent messageContent in threadMessage.ContentItems)
{
    if (messageContent is MessageTextContent textItem)
        Console.WriteLine($"\t Content: {textItem.Text}");
}


Expected output:

```
Run status: completed
Message created by/at: assistant - <some date>
	 Content: As of my last knowledge update in April 2023, the three biggest cloud providers based on global regional presence are:
     ...
```

## Step 7: Clean up resources

Assistant as well as Thread will be deleted. 

In [ ]:
await assistantsClient.DeleteAssistantAsync(responseAssistant.Value.Id);
await assistantsClient.DeleteThreadAsync(responseThread.Value.Id);

Console.WriteLine($"Assistant and Thread deleted...");

        

Expected output:

```
Assistant and Thread deleted ...
```
